<a href="https://colab.research.google.com/github/codebybishwa/Attendance-Tracker/blob/main/Modified_Attendance_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill
from datetime import datetime, timedelta

In [36]:
import re

# Function to read and process the files
def read_files():

    with open('/content/stud_list.txt', 'r') as file:
        students = [line.strip() for line in file.readlines()]

    with open('/content/python dates.txt', 'r') as file:
        content = file.read()

    # Fixing any class_timing without quotes using regex
    content = re.sub(r'class_timing\s*=\s*([0-9]{2}:[0-9]{2})\s*-\s*([0-9]{2}:[0-9]{2})',
                     r'class_timing = "\1 - \2"', content)

    # Creating a dictionary to store the variables from python dates.txt
    date_vars = {}

    try:
        exec(content, {}, date_vars)
    except SyntaxError as e:
        print(f"Error in executing content: {e}")
        return None, None, None, None

    # Extracting the values from date_vars
    lecture_dates = date_vars.get("classes_taken_dates", [])
    missed_dates = date_vars.get("classes_missed_dates", [])
    exam_dates = date_vars.get("exams_dates", [])
    class_timing = date_vars.get("class_timing", "18:00 - 20:00")  # Default value if not present

    attendance = pd.read_csv('/content/input_attendance.csv', parse_dates=['Timestamp'])

    return students, lecture_dates, attendance, class_timing

students, lecture_dates, attendance, class_timing = read_files()

# Printing results to inspect data
print("Students List (First 5):", students[:5])
print("Lecture Dates:", lecture_dates)
print("Class Timing:", class_timing)
print("Attendance Data (First 5 rows):")
print(attendance.head())

Students List (First 5): ['2201CB05 Ade Balakrishna', '2201CB11 Anvaya Sharma', '2201CB19 Chhavi Bamoriya', '2201CB21 Dharmraj Dhaker', '2201CB25 Harshita Singh']
Lecture Dates: ['06/08/2024', '13/08/2024', '20/08/2024', '27/08/2024', '03/09/2024', '17/09/2024', '01/10/2024']
Class Timing: 18:00 - 20:00
Attendance Data (First 5 rows):
             Timestamp                           Roll
0  06/08/2024 18:18:23           2201MM26 Rudra Goyal
1  06/08/2024 18:18:26  2201CE30 Lakshya Pratap Singh
2  06/08/2024 18:18:26       2201CB19 Chhavi Bamoriya
3  06/08/2024 18:18:28          2201CE31 Mayank Kumar
4  06/08/2024 18:18:31         2201CB35 Nakka Supraja


In [50]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from datetime import datetime


def clean_class_timing(class_timing):
    """
    Ensures that class_timing is a valid string and handles cases where the double quotes are missing.
    Returns a cleaned class_timing string with the proper format 'HH:MM - HH:MM'.
    """
    # Remove any non-numeric, colon, or dash characters at the ends (like spaces or quotes)
    class_timing = class_timing.strip()

    if " - " not in class_timing:
        raise ValueError(f"Invalid class_timing format: {class_timing}")

    return class_timing

def process_attendance(students, lecture_dates, attendance, class_timing):
    """
    Processes attendance data, caps at 2 per class, tracks proxies, and returns
    - attendance_record: capped at 2 per day,
    - proxy_record: number of proxies,
    - total_classes: the total number of classes (2 per day).
    """
    class_timing = clean_class_timing(class_timing)

    start_str, end_str = class_timing.split(" - ")
    start_time = datetime.strptime(start_str, "%H:%M").time()
    end_time = datetime.strptime(end_str, "%H:%M").time()

    attendance['Timestamp'] = pd.to_datetime(attendance['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='raise')

    # Initializing records
    attendance_record = {student: {date: 0 for date in lecture_dates} for student in students}
    proxy_record = {student: {date: 0 for date in lecture_dates} for student in students}
    total_classes = len(lecture_dates) * 2  # Each date has 2 classes

    for lecture_date in lecture_dates:
        lecture_start_str = f"{lecture_date} {start_time.strftime('%H:%M:%S')}"
        lecture_end_str = f"{lecture_date} {end_time.strftime('%H:%M:%S')}"

        try:
            lecture_start = datetime.strptime(lecture_start_str, '%d/%m/%Y %H:%M:%S')
            lecture_end = datetime.strptime(lecture_end_str, '%d/%m/%Y %H:%M:%S')
        except ValueError as e:
            print(f"Error parsing date: {lecture_date} - {e}")
            continue

        # Filter attendance between class timing
        lecture_attendance = attendance[(attendance['Timestamp'] >= lecture_start) & (attendance['Timestamp'] <= lecture_end)]

        for student_roll in lecture_attendance['Roll'].unique():
            student_entries = lecture_attendance[lecture_attendance['Roll'] == student_roll]
            attendance_count = len(student_entries)

            # Cap attendance at 2 and treat extra as proxies
            if student_roll in attendance_record:
                if attendance_count > 2:
                    attendance_record[student_roll][lecture_date] = 2  # Cap attendance at 2
                    proxy_record[student_roll][lecture_date] = attendance_count - 2  # Proxies are extra
                else:
                    attendance_record[student_roll][lecture_date] = attendance_count

    return attendance_record, proxy_record, total_classes


def calculate_summary(attendance_record, proxy_record, lecture_dates, total_classes):
    """
    Calculates the attendance summary for each student including missed, attended, percentage attended,
    and percentage missed. Proxies are excluded from these calculations.
    """
    summary = {}

    for student, attendance_data in attendance_record.items():
        attended_classes = sum(attendance_data[date] for date in lecture_dates)  # Count only valid attendances (max 2 per day)
        missed_classes = total_classes - attended_classes  # Calculate missed classes
        percentage_attended = (attended_classes / total_classes) * 100
        percentage_missed = (missed_classes / total_classes) * 100
        total_proxy = proxy_record[student]  # Proxy is counted separately

        summary[student] = {
            'missed': missed_classes,
            'attended': attended_classes,
            'percentage_attended': percentage_attended,
            'percentage_missed': percentage_missed,
            'proxy': total_proxy
        }

    return summary

In [51]:
# Process the attendance and output the intermediate attendance record
attendance_record, proxy_record, total_classes = process_attendance(students, lecture_dates, attendance, class_timing) # Unpack the returned tuple into three variables

# Calculate summary data using proxy_record and total_classes
summary_data = calculate_summary(attendance_record, proxy_record, lecture_dates, total_classes)

# Output intermediate data for checking
print("\nAttendance Record (First 5 Students):")
for student, record in list(attendance_record.items())[:5]:
    print(f"{student}: {record}")


Attendance Record (First 5 Students):
2201CB05 Ade Balakrishna: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 0, '01/10/2024': 2}
2201CB11 Anvaya Sharma: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 2}
2201CB19 Chhavi Bamoriya: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 2}
2201CB21 Dharmraj Dhaker: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 1, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 1}
2201CB25 Harshita Singh: {'06/08/2024': 1, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 2}


In [52]:
def generate_excel(attendance_record, proxy_record, summary_data, lecture_dates):
    """
    Generates an Excel file with:
    - attendance details per day,
    - proxy column,
    - summary data (attended, missed, percentage attended/missed).
    """
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = 'Attendance Record'

    # Header
    ws.cell(row=1, column=1, value='Roll No.')
    for i, date in enumerate(lecture_dates, start=2):
        ws.cell(row=1, column=i, value=date)
    ws.cell(row=1, column=len(lecture_dates) + 2, value='Proxy')
    ws.cell(row=1, column=len(lecture_dates) + 3, value='Missed')
    ws.cell(row=1, column=len(lecture_dates) + 4, value='Attended')
    ws.cell(row=1, column=len(lecture_dates) + 5, value='Percentage Attended')
    ws.cell(row=1, column=len(lecture_dates) + 6, value='Percentage Missed')

    # Color fill settings
    fill_absent = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # Red for absent
    fill_partial = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow for partial
    fill_full = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")    # Green for full

    for row_idx, student_roll in enumerate(attendance_record.keys(), start=2):
        ws.cell(row=row_idx, column=1, value=student_roll)  # Roll number

        for col_idx, date in enumerate(lecture_dates, start=2):
            attendance_status = attendance_record[student_roll][date]
            proxy_count = proxy_record[student_roll][date]

            # Write attendance status and color cell
            if attendance_status == 0:
                cell_value = "0"
                fill_color = fill_absent
            elif attendance_status == 1:
                cell_value = "1"
                fill_color = fill_partial
            else:
                cell_value = f"{attendance_status + proxy_count}"  # Include proxy in the cell value
                fill_color = fill_full

            cell = ws.cell(row=row_idx, column=col_idx, value=cell_value)
            cell.fill = fill_color

        # Write proxy count for each student
        ws.cell(row=row_idx, column=len(lecture_dates) + 2, value=sum(proxy_record[student_roll].values()))

        # Write summary data (missed, attended, percentages)
        ws.cell(row=row_idx, column=len(lecture_dates) + 3, value=summary_data[student_roll]['missed'])
        ws.cell(row=row_idx, column=len(lecture_dates) + 4, value=summary_data[student_roll]['attended'])
        ws.cell(row=row_idx, column=len(lecture_dates) + 5, value=summary_data[student_roll]['percentage_attended'])
        ws.cell(row=row_idx, column=len(lecture_dates) + 6, value=summary_data[student_roll]['percentage_missed'])

    wb.save('output_excel.xlsx')

In [53]:
students, lecture_dates, attendance, class_timing = read_files()

In [54]:
attendance_record, proxy_record, total_classes = process_attendance(students, lecture_dates, attendance, class_timing)

In [55]:
summary_data = calculate_summary(attendance_record, proxy_record, lecture_dates, total_classes)

In [56]:
generate_excel(attendance_record, proxy_record, summary_data, lecture_dates)